In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기

In [ ]:
# 1) 데이터 로드
iris = load_iris()
X = iris.data      # (150, 4)
y = iris.target    # (150,)  -> 0,1,2 (세 종류의 꽃)

In [ ]:
print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# 2) train / test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# 4) torch tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # CrossEntropyLoss는 long 라벨 필요
y_test  = torch.tensor(y_test,  dtype=torch.long)

In [ ]:
# 5) Dataset & DataLoader
train_ds = TensorDataset(X_train, y_train)
test_ds  = TensorDataset(X_test,  y_test)

In [ ]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=16, shuffle=False)

# DNN 모델 정의

In [ ]:
class IrisNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.Sigmoid(),
            nn.Linear(16, 16),
            nn.Sigmoid(),
            nn.Linear(16, 3)  # 클래스 3개
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
model = IrisNet()

In [ ]:
criterion = nn.CrossEntropyLoss()           # 분류용 손실
optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 루프

In [ ]:
num_epochs = 100

In [ ]:
# 순전파 > 손실계산 > 역전파 > 가중치 업데이트 흐름으로 num_epochs 만큼 학습.
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        # 1) 순전파
        outputs = model(X_batch)          # (batch_size, 3)

        # 2) 손실 계산
        loss = criterion(outputs, y_batch)

        # 3) gradient 초기화 & 역전파
        optimizer.zero_grad()
        loss.backward()

        # 4) 가중치 업데이트
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1:3d}/{num_epochs}]  Loss: {epoch_loss:.4f}")

# 학습된 모델 테스트

In [ ]:
model.eval()
correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)              # (batch_size, 3)
        _, predicted = torch.max(outputs, 1)  # 가장 큰 값의 인덱스를 예측 클래스라고 봄
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")